# Source and Location Practice

M&I 4e

Ch 13 Electric Field

- §13.X &ndash; Student Resources

In [ ]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]
>
>   A particle with charge $+5$ [nC] (a nano-coulomb is $1 \times 10^{-9}$ [C]) 
>   is located at the origin. 
>
>   __What is the electric field due to this particle, at a location 
>   $\langle 0.4, 0, 0\rangle$ [m]?__
>
>   &emsp;&emsp;&emsp;&emsp; 
>   $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]
>
>   __Lithium nucleus affected by an electric field__
>
>   A proton at location $A$ makes an electric field $\vec{E}_1$ at location $B$.
>   A different proton, placed at location $B$, experiences a force $\vec{F}_1$.
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='2a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(A)
>
>   __If $\lvert \vec{E}_1 \rvert = 500$ [N/C], what is $\lvert \vec{F}_1 \rvert$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\lvert \vec{F}_1 \rvert = \fbox{\hspace{8em}?}$&emsp;[N/C]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='2b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(B)
>
>   Now the proton at location $B$ is removed, and replaced by a lithium nucleus,
>   containing three protons and four neutrons. The proton at location $A$ 
>   remains in place. 
>
>   __What is the magnitude of the electric force on the lithium nucleus?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\lvert \vec{F}_{\mathrm{on\ Li}} \rvert = \fbox{\hspace{8em}?}$&emsp;[N]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='2c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(C)
>
>   Now the Lithium nucleus is removed, and an electron is placed at location $B$.
>   The proton at location $A$ remains in place.
>
>   __What is the magnitude of the electric force on the electron?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\lvert \vec{F}_{\mathrm{on\ e^-}} \rvert = \fbox{\hspace{8em}?}$&emsp;[N]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='2d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(D)
>
>   __Which arrow below best indicates the direction of the force on the electron, due to the electric field?__
>
>   
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]
>
>   You want to calculate the electric field at location 
>   $\langle 0.5, -0.6, 0.5 \rangle$&ensp;[m], due to a particle with charge 
>   $+7.5$ [nC] located at $\langle -0.7, -0.7, -0.2\rangle$&ensp;[m].
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='3a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the source location?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r}_\textrm{source} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the observation location?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r}_\textrm{obs} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the vector $\vec{r}$, that points from the source location to the 
>   observation location?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is $\lvert \vec{r} \rvert$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}$&emsp;[m]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What is the vector $\hat{r}$?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\hat{r} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3f' style="display:none">(F)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(F)
>
>   __What is the value of $\frac{1}{4\pi\epsilon_0} \frac{q}{ {\lvert \vec{r} \rvert}^2 }$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}$&emsp;[N/C]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3g' style="display:none">(G)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(G)
>
>   Finally, __what is the electric field, expressed as a vector?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]
>
>   __What is the electric field__, at a location $\vec{b} =\langle -0.1, -0.3, 0\rangle$ [m],
>   due to a particle with charge $+2$ [nC] located at the origin?
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]</h2>

>   ---
>   ## [5]
>
>   __What are the magnitude and direction of the electric field $\vec{E}$,__ at
>   a location $\langle 32, 0, 0\rangle$ [cm]; if there is a negative point
>   charge of $2.5$ [nC] ($2.5 \times 10^{-9}$ [C]) at location 
>   $\langle 45, 0, 0\rangle$ [cm]?
>
>   Include units. Express your answer in vector form. 
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle$
>
>   (select one:)
>
>   - [ ] [N/C]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

- [x] [N/C]

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [6]
>
>   An electron is located at $\langle 0.7, 0.7, -0.2\rangle$ [m]. You need to 
>   find the electric field at location $\langle 0.5, 1.7, -0.5\rangle$ [m] due
>   to the electron. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the source location?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r}_\textrm{source} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the observation location?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r}_\textrm{obs} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the vector $\vec{r}$ that points from the source location, to the
>   observation location?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is $\lvert\vec{r}\rvert$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\fbox{\hspace{8em}?}\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What is the vector $\hat{r}$?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\hat{r} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6f' style="display:none">(F)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(F)
>
>   __What is the value of $\frac{1}{4\pi\epsilon_0} \frac{q}{ {\lvert \vec{r} \rvert}^2 }$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}$&emsp;[N/C]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6g' style="display:none">(G)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(G)
>
>   Finally, __what is the electric field, expressed as a vector?__
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]</h2>

>   ---
>   ## [7]
>
>   The electric field at a location $C$ points north, and the magnitude is
>   $4.4 \times 10^6$ [N/C]. 
>
>   Give numerical answers to the following questions.
>
>   (Assume that the $+x$-axis is to the east, the $+y$-axis points to the north,
>   and the $+z$-axis points towards you out of the page. Express your answers
>   in vector form.)
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Where, relative to $C$, should you place a single proton to produce this field?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r}_p = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Where, relative to $C$, should you place a single electron to produce this field?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r}_e = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(C)
>
>   __Where should you place a proton and an electron, at equal distances to $C$, to produce this field?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r}_p = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r}_e = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   ---
>   ## [8]
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='8a' style="display:none">(A)</h3>
<h4 id='8a_i' style="display:none">{A.ii}</h3>


>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   #### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;{A.i}
>
>   On a clear and carefully drawn diagram, place a helium nucleus (consisting 
>   of two protons and two neutrons) and a proton, in such a way that the 
>   electric field due to these charges is zero at a location marked $X$; a 
>   distance $7 \times 10^{-10}$ [m] from the helium nucleus.
>
>   (Assume the $+x$-axis points to the right.)
>
>   (select one:)
>
>   - [ ] 
>
>   - [ ] 
>
>   - [ ] 
>
>   - [ ] 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h4 id='8a_ii' style="display:none">{A.ii}</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;{A.ii}
>
>   __Be quantitative about the relative distances. (Enter the position of the 
>   proton relative to the location marked $X$).__
>
>   Express your answer in vectorm form.
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='8b' style="display:none">(B)</h3>
<h4 id='8b_i' style="display:none">{B.i}</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   #### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;{B.i}
>
>   On a clear and carefully drawn diagram, place a helium nucleus  and an 
>   _electron_, in such a way that the electric field due to these charges is 
>   zero at a location marked $X$.
>
>   (Assume the $+x$-axis points to the right.)
>
>   (select one:)
>
>   - [ ] 
>
>   - [ ] 
>
>   - [ ] 
>
>   - [ ] 
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h4 id='8b_ii' style="display:none">{B.ii}</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;{B.ii}
>
>   __Be quantitative about the relative distances. (Enter the position of the 
>   electron relative to the location marked $X$).__
>
>   Express your answer in vectorm form.
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{r} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='9' style="display:none">[9]</h2>

>   ---
>   ## [9]
>
>   
>
>   Two small charged objects are located as shown in the diagram. In the 
>   diagram, each square represents $10^{-2}$ [m]. 
>   $Q_1 = -2$ [nC] and $Q_2 = 5$ [nC].
>   
>   Remember that $1$ [nC] (nano-coulomb) is $10^{-9}$ coulombs. 
>
>   You will calculate the net electric field at a location 
>   $\vec{A} = \langle -0.04, -0.06, 0\rangle$ [m]. 
>
>   Before doing any calculations, on a copy of the diagram, draw two arrows
>   representing the electric field at the observation location due to each of
>   the charged particles. Use your diagram to answer the following questions.
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='9a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the sign of the $x$-component of $\vec{E}_1$, the electric field at $\vec{A}$ due to $Q_1$?__
>
>   - [ ] positive 
>
>   __What is the sign of the $y$-component of $\vec{E}_1$?__
>
>   - [ ] positive 
>
>   __What is the sign of the $z$-component of $\vec{E}_1$?__
>
>   - [ ] zero
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] positive
- [x] positive
- [x] zero

<h3 id='9b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the sign of the $x$-component of $\vec{E}_2$, the electric field at $\vec{A}$ due to $Q_2$?__
>
>   - [ ] negative
>
>   __What is the sign of the $y$-component of $\vec{E}_2$?__
>
>   - [ ] negative
>
>   __What is the sign of the $z$-component of $\vec{E}_2$?__
>
>   - [ ] zero
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] negative
- [x] negative
- [x] zero

<h3 id='9c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the electric field at $\vec{A}$ due to $Q_1$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E}_1 = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the electric field at $\vec{A}$ due to $Q_2$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E}_2 = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What is the electric field at $\vec{A}$?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E}_\textrm{net} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='10' style="display:none">[10]</h2>

>   ---
>   ## [10]
>
>   __Field and force with three charges__
>
>   At a particular moment, one negative and two positive charges are located as
>   shown in the figure. 
>
>   
>
>   Your answers to each part of this problem should be vectors.
>
>   It helps a great deal to make a diagram with arrows representing the various
>   electric field contributions, and then check the signs of your components 
>   against these arrows. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='10a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Find the elctric field at the location of $Q_1$, due to $Q_2$ and $Q_3$.__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='10b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Use the electric field you calculated in part ([A](#10a)) to find the force on $Q_1$.__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{F} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='10c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(C)
>
>   __Find the elctric field at the location $A$, due to all three charges.__
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='10d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(D)
>
>   An alpha particle ($\mathrm{He^{2+}}$, containing two protons and two neutrons)
>   is released from rest at location $A$. 
>
>   Use your answer from part ([C](#10c)) to __determine the initial acceleration
>   of the alpha particle.
>
>   (Use $6.646 \times 10^{-27}$ [kg] for the mass of $\mathrm{He^{2+}}$.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{a} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{m / s^2}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='11' style="display:none">[11]</h2>

>   ---
>   ## [11]
>
>   A dipole consists of two charges $+q$ and $-q$, held apart by a rod of length
>   $s$, as shown in the diagram. 
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='11a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B9;&hairsp;&#x2E05;&hairsp;(A)
>
>   If $q=3$ [nC] and $s=1$ [mm], __what is the magnitude of the electric field due to the dipole__
>   at location $A$, a distance $d=5$ [cm] from the dipole?
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $E = \fbox{\hspace{8em}?}\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='11a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B9;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the magnitude of the electric field due to the dipole__ at location
>   $B$, a distance $d=5$ [cm] from the dipole?
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $E = \fbox{\hspace{8em}?}\ \left[\mathrm{N/C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='12' style="display:none">[12]</h2>

>   ---
>   ## [12]
>
>   Two dipoles are oriented as shown in the figure below.
>
>   
>
>   Each dipole consists of charges held apart by a short rod (not shown to scale).
>
>   __What is the electric field at location $A$?__
>
>   Start by drawing a diagram that shows the direction of each dipole's 
>   contribution to the electric field at location $A$. Assume $Q_1 = 19$ [nC]
>   and $Q_2 = 8$ [nC].
>
>   (Express your answer in vector form.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $\vec{E}_A = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle\ \left[\mathrm{N/C}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='13' style="display:none">[13]</h2>

>   ---
>   ## [13]
>
>   A charge of $+1$ [nC] ($1 \times 10^{-9}$ [C]) and a dipole with charges
>   $+q$ and $-q$, separated by $0.3$ [mm], contribute a net field at location 
>   $A$ that is zero, as shown in the figure below. (Assume $r_1 = 64$ [cm] and
>   $r_2 = 36$ [cm].)
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='13a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B3;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Which end of the dipole is positively charged?__
>
>   - [ ] the left end
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] the left end

<h3 id='13b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x00B3;&hairsp;&#x2E05;&hairsp;(B)
>
>   How large is the charge $q$?
>
>   (Enter the absolute value.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp;
>   $q = \fbox{\hspace{8em}?}\ \left[\mathrm{C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

---